In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_file = './spvae_vectors_drags.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_120,dim_121,dim_122,dim_123,dim_124,dim_125,dim_126,dim_127,dim_128,drag
61,-0.996583,0.253377,2.345441,-0.989086,-2.179726,4.450816,-0.333054,-0.833110,0.170525,0.681121,...,1.321793,-0.977492,0.493258,-2.447803,-0.851386,0.003659,-0.704168,0.301570,0.179546,0.375
354,-0.107085,1.786517,1.586284,-1.288352,-1.469519,1.856284,0.022899,0.707441,0.195338,0.934119,...,0.785785,-1.589034,-3.525021,-4.223967,-2.653476,0.012422,0.597222,0.977202,-0.662360,0.374
358,-3.295154,-1.133777,-0.713116,0.946432,0.027521,0.023506,0.565126,-1.224105,-0.519147,0.146645,...,2.272549,-0.996835,5.970934,0.562599,3.063003,-0.003317,1.284646,-0.388674,0.367492,0.435
275,-0.809618,-1.043127,0.594460,1.935878,0.173827,1.941216,-0.689809,-1.535557,1.745006,0.349334,...,0.602183,-0.519529,0.047703,1.788710,-0.448435,-0.003273,-0.998566,-0.302207,0.684875,0.437
18,-0.572277,-0.328782,0.444743,0.411545,-1.628947,3.242414,-0.008619,-0.790840,0.288297,0.432827,...,2.103731,-2.167353,-1.278066,-1.638192,2.981116,0.002460,-0.616031,-0.505367,0.432376,0.367


In [2]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    351.000000
mean       0.398513
std        0.060013
min        0.278000
25%        0.353000
50%        0.394000
75%        0.435000
max        0.598000
Name: drag, dtype: float64


In [3]:
import os
save_path = './agModels-spvae'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, presets='best_quality', num_bag_folds=10)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=10, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./agModels-spvae/"
AutoGluon Version:  0.7.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #51-Ubuntu SMP Mon Jul 4 06:41:22 UTC 2022
Train Data Rows:    351
Train Data Columns: 128
Label Column: drag
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (0.598, 0.278, 0.39851, 0.06001)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:    

In [4]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_119,dim_120,dim_121,dim_122,dim_123,dim_124,dim_125,dim_126,dim_127,dim_128
46,-1.675357,0.774387,-2.380979,-0.728630,-1.412328,1.705713,-0.913276,-0.664502,-0.036643,0.697300,...,2.785418,-0.100383,-1.418045,-5.795298,-0.239288,-3.270577,0.005218,2.296032,-0.461153,1.085652
101,-0.058659,0.094974,-0.148768,-1.709858,0.231730,0.003423,-0.203162,0.288094,-0.420197,-2.209749,...,-2.907042,0.220606,0.879496,-1.250059,-0.911256,2.207671,-0.007707,1.828314,-0.745860,-1.381517
175,0.408514,-0.318794,-0.195711,-1.258788,-0.074261,-3.524118,0.243094,-0.282567,0.518098,1.104262,...,-2.164850,-2.873618,0.817046,1.877096,-1.402977,-0.992778,-0.002622,1.483371,0.380046,0.484612
9,-2.754269,-1.133549,-1.814455,0.036195,-0.159687,2.911953,-0.640023,-0.894014,0.440514,0.164435,...,-2.664808,1.762599,-0.071357,3.233031,-0.915575,2.781991,-0.008800,0.508250,-0.724321,-0.320688
136,-1.054854,0.100548,0.473871,3.140375,-0.449581,2.196428,0.303511,-0.011313,0.375206,-0.866014,...,3.855908,0.273155,-2.094910,-3.537341,1.143077,-2.242801,-0.000889,0.217993,-0.597952,1.817741


In [6]:
%%capture log_output
%config InlineBackend.figure_format = 'retina'
%config Application.log_level = 'DEBUG'
%config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

with open('./output_spvae.log', 'w') as f:
    f.write(log_output.stdout)

Evaluation: root_mean_squared_error on test data: -0.05345021195429395
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -0.05345021195429395,
    "mean_squared_error": -0.0028569251579589477,
    "mean_absolute_error": -0.04031433976780285,
    "r2": 0.028458675503590602,
    "pearsonr": 0.2177279680964373,
    "median_absolute_error": -0.028789700865745532
}


In [ ]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', []) : 64 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [ ]:
predictor.predict(test_data, model='XGBoost_BAG_L1')

46     0.382149
101    0.389454
175    0.431892
9      0.384073
136    0.390649
         ...   
173    0.394211
5      0.402515
55     0.423535
428    0.396295
334    0.420224
Name: drag, Length: 88, dtype: float32